In [1]:

import pandas as pd
import numpy as np
import typing

import sys
sys.path.append('..')
import doctable

In [2]:
@doctable.table_schema
class Record:
    name: str = doctable.Column(column_args=doctable.ColumnArgs(nullable=False, unique=True))
    age: int = doctable.Column()
    is_old: bool = doctable.Column()
    
    id: int = doctable.Column(
        column_args=doctable.ColumnArgs(
            order = 0, 
            primary_key=True, 
            autoincrement=True
        ),
    )

core = doctable.ConnectCore.open(target=':memory:', dialect='sqlite', echo=True)

with core.begin_ddl() as ddl:
    rtab = ddl.create_table_if_not_exists(container_type=Record)

2023-11-12 13:03:46,351 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-12 13:03:46,352 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Record")
2023-11-12 13:03:46,352 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-11-12 13:03:46,353 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Record")
2023-11-12 13:03:46,353 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-11-12 13:03:46,355 INFO sqlalchemy.engine.Engine 
CREATE TABLE "Record" (
	id INTEGER, 
	age INTEGER, 
	is_old INTEGER, 
	name VARCHAR NOT NULL, 
	PRIMARY KEY (id), 
	UNIQUE (name)
)


2023-11-12 13:03:46,355 INFO sqlalchemy.engine.Engine [no key 0.00040s] ()
2023-11-12 13:03:46,356 INFO sqlalchemy.engine.Engine COMMIT


In [3]:
import random
random.seed(0)

new_records: typing.List[Record] = list()
for i in range(10):
    age = int(random.random()*100) # number in [0,1]
    is_old = age > 50
    new_records.append(Record(name='user_'+str(i), age=age, is_old=is_old))

# insert new records
with rtab.query() as q:
    print(q.insert_multi(new_records))

# dataframe select (for example purposes - .df() will be covered later)
with core.query() as q:
    r = q.select(rtab.all_cols()).df()
r

2023-11-12 13:03:46,396 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-12 13:03:46,397 INFO sqlalchemy.engine.Engine INSERT OR FAIL INTO "Record" (age, is_old, name) VALUES (?, ?, ?)
2023-11-12 13:03:46,397 INFO sqlalchemy.engine.Engine [generated in 0.00147s] [(84, True, 'user_0'), (75, True, 'user_1'), (42, False, 'user_2'), (25, False, 'user_3'), (51, True, 'user_4'), (40, False, 'user_5'), (78, True, 'user_6'), (30, False, 'user_7'), (47, False, 'user_8'), (58, True, 'user_9')]
2023-11-12 13:03:46,398 INFO sqlalchemy.engine.Engine COMMIT
2023-11-12 13:03:46,400 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-12 13:03:46,400 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".age, "Record".is_old, "Record".name 
FROM "Record"
2023-11-12 13:03:46,401 INFO sqlalchemy.engine.Engine [generated in 0.00095s] ()
2023-11-12 13:03:46,403 INFO sqlalchemy.engine.Engine COMMIT


,id,age,is_old,name
0,1,84,1,user_0
1,2,75,1,user_1
2,3,42,0,user_2
3,4,25,0,user_3
4,5,51,1,user_4
5,6,40,0,user_5
6,7,78,1,user_6
7,8,30,0,user_7
8,9,47,0,user_8
9,10,58,1,user_9


## Two Interfaces: `ConnectQuery` and `TableQuery`

There are two interfaces for performing queries: `ConnectQuery` and `TableQuery`. 

+ **`ConnectQuery`** table-agnostic interface for querying any table in any result format.

+ **`TableQuery`** table-specific interface for querying a specific table. Insert and select from container objects used to define the schema.


In [4]:
# ConnectQuery - table agnostic
with core.query() as q:
    print(type(q))

# TableQuery - queries are relative to specific 
# table, results appear as container objects
with rtab.query() as q:
    print(type(q))

<class 'doctable.query.connectquery.ConnectQuery'>
<class 'doctable.query.tablequery.TableQuery'>


### `ConnectQuery` Basics

Every query begins with the `.query()` method, which returns a `ConnectQuery` object. This object maintains a database connection, and, when used as a context manager, will commit all changes upon exit. It is fine to use the `ConnectQuery` object without a context manager for queries that do not require commits.

This example is the most basic select query we can execute. Note that `ConnectQuery` methods are table-agnostic, so we must specify columns to be selected - in this case, we provide `rtab.all_cols()` to specify that we want to query all columns from the `Record` table. It returns a `sqlalchemy.CursorResult` object that we will discuss later.

In [5]:
core.query().select(rtab.all_cols())

2023-11-12 13:03:46,500 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-12 13:03:46,500 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".age, "Record".is_old, "Record".name 
FROM "Record"
2023-11-12 13:03:46,501 INFO sqlalchemy.engine.Engine [cached since 0.101s ago] ()


### Selecting Specific Columns

In many cases, you would not want to select all columns from a given table - for this reason, there are several methods you can use to specify the desired columns. In addition to `.all_cols()` in the above snippet, you may use any of these methods.

| Method | Description |
| --- | --- |
| `.all_cols()` | specify that we want all columns |
| `.cols('col1', 'col2')` | specify set of columns |
| `table['col1']` | specify single column |
| `table[['col1', 'col2']]` | specify multiple columns |
| `table['col1':'col3']` | specify sequential range of columns |

In [6]:
# select all columns
rtab.all_cols()

[Column('id', Integer(), table=<Record>, primary_key=True),
 Column('age', Integer(), table=<Record>),
 Column('is_old', Integer(), table=<Record>),
 Column('name', String(), table=<Record>, nullable=False)]

In [7]:
# .cols method
rtab.cols('id', 'name')

[Column('id', Integer(), table=<Record>, primary_key=True),
 Column('name', String(), table=<Record>, nullable=False)]

In [8]:
# single-column subscript
rtab['age']

Column('age', Integer(), table=<Record>)

In [9]:
# list of columns
rtab[['id','is_old']]

[Column('id', Integer(), table=<Record>, primary_key=True),
 Column('is_old', Integer(), table=<Record>)]

In [10]:
# slice select
rtab['id':'is_old']

[Column('id', Integer(), table=<Record>, primary_key=True),
 Column('age', Integer(), table=<Record>),
 Column('is_old', Integer(), table=<Record>)]

Note that the `.select()` method requires a list of columns, so we can combine these methods by combining the lists they return. Obviously, the order matters for the returned values.

In [11]:
rtab.cols('id','is_old') + [rtab['name']]

[Column('id', Integer(), table=<Record>, primary_key=True),
 Column('is_old', Integer(), table=<Record>),
 Column('name', String(), table=<Record>, nullable=False)]

The `.select()` method always accepts a list of columns, so be sure to wrap single-column selections in a list.

In [12]:
core.query().select([rtab['name']])

2023-11-12 13:03:46,868 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-12 13:03:46,868 INFO sqlalchemy.engine.Engine SELECT "Record".name 
FROM "Record"
2023-11-12 13:03:46,869 INFO sqlalchemy.engine.Engine [generated in 0.00143s] ()


In [13]:
core.query().select(rtab.cols('id','is_old') + [rtab['name']])

2023-11-12 13:03:46,916 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-12 13:03:46,917 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".is_old, "Record".name 
FROM "Record"
2023-11-12 13:03:46,918 INFO sqlalchemy.engine.Engine [generated in 0.00134s] ()


### Working with Query Results

Now we turn to working with the results objects. So far I have demonstrated values for returning `sqlalchemy.CursorResult` objects, but additional methods are required to return the results in a usable format. The following methods are available for various purposes:

| Method | Description |
| --- | --- |
| `result.all()` | return all results in query
| `result.df()` | return multiple results as a dataframe
| `result.first()` | return first result in query
| `result.one()` | return exactly one result in query. NOTE: raises exception if not exactly one result.
| `result.scalar_one()` | return single result, end query. NOTE: raises exception if not exactly one result.
| `result.scalars().all()` | return single column of results

In [14]:
with core.query() as q:
    r = q.select(rtab.all_cols(), limit=3)
r.all()

2023-11-12 13:03:46,966 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-12 13:03:46,967 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".age, "Record".is_old, "Record".name 
FROM "Record"
 LIMIT ? OFFSET ?
2023-11-12 13:03:46,968 INFO sqlalchemy.engine.Engine [generated in 0.00149s] (3, 0)
2023-11-12 13:03:46,969 INFO sqlalchemy.engine.Engine COMMIT


[(1, 84, 1, 'user_0'), (2, 75, 1, 'user_1'), (3, 42, 0, 'user_2')]

In [15]:
core.query().select(rtab.all_cols(), limit=3).df()

2023-11-12 13:03:47,020 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-12 13:03:47,021 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".age, "Record".is_old, "Record".name 
FROM "Record"
 LIMIT ? OFFSET ?
2023-11-12 13:03:47,021 INFO sqlalchemy.engine.Engine [cached since 0.05495s ago] (3, 0)


,id,age,is_old,name
0,1,84,1,user_0
1,2,75,1,user_1
2,3,42,0,user_2


In [16]:
# raises exception without limit=1
core.query().select(rtab.all_cols()).first()

2023-11-12 13:03:47,068 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-12 13:03:47,069 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".age, "Record".is_old, "Record".name 
FROM "Record"
2023-11-12 13:03:47,069 INFO sqlalchemy.engine.Engine [cached since 0.6691s ago] ()


(1, 84, 1, 'user_0')

In [17]:
# raises exception if more than one result is returned 
# (here I forced this with limit=1)
core.query().select(rtab.all_cols(), limit=1).one()

2023-11-12 13:03:47,116 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-12 13:03:47,117 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".age, "Record".is_old, "Record".name 
FROM "Record"
 LIMIT ? OFFSET ?
2023-11-12 13:03:47,117 INFO sqlalchemy.engine.Engine [cached since 0.151s ago] (1, 0)


(1, 84, 1, 'user_0')

In [18]:
# this returns the first column from the first row, then closes the cursor
core.query().select(rtab.all_cols(), limit=1).scalar_one()

2023-11-12 13:03:47,164 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-12 13:03:47,165 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".age, "Record".is_old, "Record".name 
FROM "Record"
 LIMIT ? OFFSET ?
2023-11-12 13:03:47,165 INFO sqlalchemy.engine.Engine [cached since 0.1989s ago] (1, 0)


1

In [19]:
# it makes more sense to query a single column
core.query().select(rtab.cols('is_old'), limit=1).scalar_one()

2023-11-12 13:03:47,212 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-12 13:03:47,213 INFO sqlalchemy.engine.Engine SELECT "Record".is_old 
FROM "Record"
 LIMIT ? OFFSET ?
2023-11-12 13:03:47,214 INFO sqlalchemy.engine.Engine [generated in 0.00167s] (1, 0)


1

In [20]:
# and when when you need a single column, use .scalars() instead of .all()
core.query().select(rtab.cols('is_old')).scalars().all()

2023-11-12 13:03:47,260 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-12 13:03:47,262 INFO sqlalchemy.engine.Engine SELECT "Record".is_old 
FROM "Record"
2023-11-12 13:03:47,262 INFO sqlalchemy.engine.Engine [generated in 0.00233s] ()


[1, 1, 0, 0, 1, 0, 1, 0, 0, 1]

### Conditional Select Statements

| operator | description |
| --- | --- |
| `&`, `doctable.exp.and_()` | and |
| `\|`, `doctable.exp.or_()` | or |
| `==` | equals |
| `!=`, `doctable.exp.not_()` | not equals |
| `>` | greater than |
| `>=` | greater than or equal to |
| `<` | less than |
| `<=` | less than or equal to |
| `in_()` | in list |
| `contains()` | check if item is substring |
| `like()` | like string |
| `ilike()` | case-insensitive like string |
| `between()`, `doctable.exp.between()` | between two values |
| `is_()` | is value |
| `isnot()` | is not value |
| `startswith()` | starts with string |

In [21]:
core.query().select(rtab.all_cols(), where=rtab['id']==2).df()

2023-11-12 13:03:47,309 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-12 13:03:47,310 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".age, "Record".is_old, "Record".name 
FROM "Record" 
WHERE "Record".id = ?
2023-11-12 13:03:47,311 INFO sqlalchemy.engine.Engine [generated in 0.00242s] (2,)


,id,age,is_old,name
0,2,75,1,user_1


In [22]:
core.query().select(rtab.all_cols(), where=rtab['id']<rtab['id']).df()

2023-11-12 13:03:47,357 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-12 13:03:47,358 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".age, "Record".is_old, "Record".name 
FROM "Record" 
WHERE "Record".id < "Record".id
2023-11-12 13:03:47,358 INFO sqlalchemy.engine.Engine [generated in 0.00166s] ()


""


In [23]:
core.query().select(rtab.all_cols(), where=(rtab['id']%2)==0).df()

2023-11-12 13:03:47,405 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-12 13:03:47,405 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".age, "Record".is_old, "Record".name 
FROM "Record" 
WHERE "Record".id % ? = ?
2023-11-12 13:03:47,406 INFO sqlalchemy.engine.Engine [generated in 0.00131s] (2, 0)


,id,age,is_old,name
0,2,75,1,user_1
1,4,25,0,user_3
2,6,40,0,user_5
3,8,30,0,user_7
4,10,58,1,user_9


In [24]:
condition = (rtab['id']>=2) & (rtab['id']<=4) & (rtab['name']!='user_2')
core.query().select(rtab.all_cols(), where=condition).df()

2023-11-12 13:03:47,453 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-12 13:03:47,454 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".age, "Record".is_old, "Record".name 
FROM "Record" 
WHERE "Record".id >= ? AND "Record".id <= ? AND "Record".name != ?
2023-11-12 13:03:47,454 INFO sqlalchemy.engine.Engine [generated in 0.00133s] (2, 4, 'user_2')


,id,age,is_old,name
0,2,75,1,user_1
1,4,25,0,user_3


In [25]:
condition = rtab['name'].in_(('user_2','user_3'))
core.query().select(rtab.all_cols(), where=condition).df()

2023-11-12 13:03:47,500 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-12 13:03:47,501 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".age, "Record".is_old, "Record".name 
FROM "Record" 
WHERE "Record".name IN (?, ?)
2023-11-12 13:03:47,502 INFO sqlalchemy.engine.Engine [generated in 0.00148s] ('user_2', 'user_3')


,id,age,is_old,name
0,3,42,0,user_2
1,4,25,0,user_3


In [26]:
core.query().select(rtab.all_cols(), where=rtab['id'].between(2,4)).df()

2023-11-12 13:03:47,549 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-12 13:03:47,550 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".age, "Record".is_old, "Record".name 
FROM "Record" 
WHERE "Record".id BETWEEN ? AND ?
2023-11-12 13:03:47,551 INFO sqlalchemy.engine.Engine [generated in 0.00173s] (2, 4)


,id,age,is_old,name
0,2,75,1,user_1
1,3,42,0,user_2
2,4,25,0,user_3


In [27]:
condition = ~(rtab['name'].in_(('user_2','user_3'))) & (rtab['id'] < 4)
core.query().select(rtab.all_cols(), where=condition).df()

2023-11-12 13:03:47,598 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-12 13:03:47,599 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".age, "Record".is_old, "Record".name 
FROM "Record" 
WHERE ("Record".name NOT IN (?, ?)) AND "Record".id < ?
2023-11-12 13:03:47,600 INFO sqlalchemy.engine.Engine [generated in 0.00134s] ('user_2', 'user_3', 4)


,id,age,is_old,name
0,1,84,1,user_0
1,2,75,1,user_1


In [28]:
condition = doctable.exp.or_(doctable.exp.not_(rtab['id']==4)) & (rtab['id'] <= 2)
core.query().select(rtab.all_cols(), where=condition).df()

2023-11-12 13:03:47,650 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-12 13:03:47,651 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".age, "Record".is_old, "Record".name 
FROM "Record" 
WHERE "Record".id != ? AND "Record".id <= ?
2023-11-12 13:03:47,651 INFO sqlalchemy.engine.Engine [generated in 0.00178s] (4, 2)


,id,age,is_old,name
0,1,84,1,user_0
1,2,75,1,user_1


In [29]:
with core.query() as q:
    ages = q.select([rtab['age']]).scalars().all()
    mean_age = sum(ages)/len(ages)
    result = q.select(rtab.all_cols(), where=rtab['age']>mean_age)
result.df()

2023-11-12 13:03:47,701 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-12 13:03:47,702 INFO sqlalchemy.engine.Engine SELECT "Record".age 
FROM "Record"
2023-11-12 13:03:47,702 INFO sqlalchemy.engine.Engine [generated in 0.00136s] ()
2023-11-12 13:03:47,704 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".age, "Record".is_old, "Record".name 
FROM "Record" 
WHERE "Record".age > ?
2023-11-12 13:03:47,704 INFO sqlalchemy.engine.Engine [generated in 0.00041s] (53.0,)
2023-11-12 13:03:47,705 INFO sqlalchemy.engine.Engine COMMIT


,id,age,is_old,name
0,1,84,1,user_0
1,2,75,1,user_1
2,7,78,1,user_6
3,10,58,1,user_9


### Column Operators

In addition to any of the methods used for conditional selects, there are several additional methods that can be used to transform columns in the select statement.

| Method | Description |
| --- | --- |
| `.label()` | rename column in result (particularly useful after transformations) |
| `.min()`, `doctable.f.min()` | max of column values |
| `.max()`, `doctable.f.max()` | max of column values |
| `.sum()`, `doctable.f.sum()` | sum of column |
| `.count()`, `doctable.f.count()` | count number of results. *NOTE*: must use `f.count()` when counting transformed columns. |
| `.distinct()`, `doctable.f.distinct()` | get distinct values |
| `/` | divide |
| `*` | multiply |
| `+` | add |
| `-` | subtract |
| `%` | modulo |
| `.concat()` | concatenate strings |

In [30]:
columns = [
    (rtab['id'] % 2).label('mod_id'), 
    rtab['name'].label('myname')
]
core.query().select(columns, where=rtab['is_old']).df()

2023-11-12 13:03:47,750 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-12 13:03:47,751 INFO sqlalchemy.engine.Engine SELECT "Record".id % ? AS mod_id, "Record".name AS myname 
FROM "Record" 
WHERE "Record".is_old
2023-11-12 13:03:47,752 INFO sqlalchemy.engine.Engine [generated in 0.00189s] (2,)


,mod_id,myname
0,1,user_0
1,0,user_1
2,1,user_4
3,1,user_6
4,0,user_9


In [31]:
formula = rtab['age'].sum() / rtab['age'].count()
core.query().select([formula]).scalar_one()

2023-11-12 13:03:47,802 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-12 13:03:47,803 INFO sqlalchemy.engine.Engine SELECT sum("Record".age) / (count("Record".age) + 0.0) AS anon_1 
FROM "Record"
2023-11-12 13:03:47,803 INFO sqlalchemy.engine.Engine [generated in 0.00157s] ()


Decimal('53.0000000000')

In [32]:
formula = rtab['age'].max() - rtab['age'].min()
core.query().select([formula]).scalar_one()

2023-11-12 13:03:47,853 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-12 13:03:47,854 INFO sqlalchemy.engine.Engine SELECT max("Record".age) - min("Record".age) AS anon_1 
FROM "Record"
2023-11-12 13:03:47,855 INFO sqlalchemy.engine.Engine [generated in 0.00141s] ()


59

In [33]:
# average age of individuals over 30
formula = rtab['age'].sum() / rtab['age'].count()
condition = rtab['age'] > 30
core.query().select([formula], where=condition).scalar_one()

2023-11-12 13:03:47,902 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-12 13:03:47,903 INFO sqlalchemy.engine.Engine SELECT sum("Record".age) / (count("Record".age) + 0.0) AS anon_1 
FROM "Record" 
WHERE "Record".age > ?
2023-11-12 13:03:47,903 INFO sqlalchemy.engine.Engine [generated in 0.00142s] (30,)


Decimal('59.3750000000')

In [34]:
# descriptive stats on age of individuals over 30
columns = [
    (rtab['age'].sum() / rtab['age'].count()).label('mean'),
    rtab['age'].max().label('max'),
    rtab['age'].min().label('min'),
]
condition = rtab['age'] > 30
core.query().select(columns, where=condition).df()

2023-11-12 13:03:47,954 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-12 13:03:47,955 INFO sqlalchemy.engine.Engine SELECT sum("Record".age) / (count("Record".age) + 0.0) AS mean, max("Record".age) AS max, min("Record".age) AS min 
FROM "Record" 
WHERE "Record".age > ?
2023-11-12 13:03:47,955 INFO sqlalchemy.engine.Engine [generated in 0.00127s] (30,)


,mean,max,min
0,59.3750000000,84,40


In [35]:
# all distinct values
formula = rtab['is_old'].distinct()
core.query().select([formula]).df()

2023-11-12 13:03:48,005 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-12 13:03:48,006 INFO sqlalchemy.engine.Engine SELECT distinct("Record".is_old) AS distinct_1 
FROM "Record"
2023-11-12 13:03:48,006 INFO sqlalchemy.engine.Engine [generated in 0.00139s] ()


,distinct_1
0,1
1,0


In [36]:
# count individuals over 30
core.query().select([doctable.f.count()], where=rtab['age']>30).scalar_one()

2023-11-12 13:03:48,053 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-12 13:03:48,054 INFO sqlalchemy.engine.Engine SELECT count(*) AS count_1 
FROM "Record" 
WHERE "Record".age > ?
2023-11-12 13:03:48,054 INFO sqlalchemy.engine.Engine [generated in 0.00144s] (30,)


8

In [37]:
# similar to previous
core.query().select([rtab['id'].count()], where=rtab['age']>30).scalar_one()

2023-11-12 13:03:48,101 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-12 13:03:48,102 INFO sqlalchemy.engine.Engine SELECT count("Record".id) AS count_1 
FROM "Record" 
WHERE "Record".age > ?
2023-11-12 13:03:48,103 INFO sqlalchemy.engine.Engine [generated in 0.00132s] (30,)


8

### Order By, Group By, Limit, Offset

More complicated queries involving ordering, grouping, limiting, and specifying offset can be specified using parameters to the `.select()` method.

| Parameter | Description |
| --- | --- |
| `limit` | limit number of results |
| `order_by` | list of columns to order by |
| `group_by` | list of columns to group by |
| `offset` | offset results by specified number |

##### Order By and Limits

In [38]:
# get the five youngest individuals in order
core.query().select(rtab.all_cols(), order_by=rtab['age'], limit=5).df()

2023-11-12 13:03:48,149 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-12 13:03:48,150 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".age, "Record".is_old, "Record".name 
FROM "Record" ORDER BY "Record".age
 LIMIT ? OFFSET ?
2023-11-12 13:03:48,151 INFO sqlalchemy.engine.Engine [generated in 0.00146s] (5, 0)


,id,age,is_old,name
0,4,25,0,user_3
1,8,30,0,user_7
2,6,40,0,user_5
3,3,42,0,user_2
4,9,47,0,user_8


In [39]:
# get the five oldest now
core.query().select(rtab.all_cols(), order_by=rtab['age'].desc(), limit=5).df()

2023-11-12 13:03:48,198 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-12 13:03:48,200 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".age, "Record".is_old, "Record".name 
FROM "Record" ORDER BY "Record".age DESC
 LIMIT ? OFFSET ?
2023-11-12 13:03:48,200 INFO sqlalchemy.engine.Engine [generated in 0.00204s] (5, 0)


,id,age,is_old,name
0,1,84,1,user_0
1,7,78,1,user_6
2,2,75,1,user_1
3,10,58,1,user_9
4,5,51,1,user_4


In [40]:
# order by is_old, but preserve order of id otherwise
order = [
    rtab['is_old'].desc(),
    rtab['id'].asc(),
]
core.query().select(rtab.all_cols(), order_by=order, limit=5).df()

2023-11-12 13:03:48,250 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-12 13:03:48,251 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".age, "Record".is_old, "Record".name 
FROM "Record" ORDER BY "Record".is_old DESC, "Record".id ASC
 LIMIT ? OFFSET ?
2023-11-12 13:03:48,251 INFO sqlalchemy.engine.Engine [generated in 0.00154s] (5, 0)


,id,age,is_old,name
0,1,84,1,user_0
1,2,75,1,user_1
2,5,51,1,user_4
3,7,78,1,user_6
4,10,58,1,user_9


#### Grouping and Column Operators

In [41]:
# summary stats by is_old
cols = [
    #rtab['is_old'].count().label('count'),
    doctable.f.count().label('count'),
    rtab['age'].min().label('min'),
    rtab['age'].max().label('max'),
    (rtab['age'].sum()/rtab['age'].count()).label('mean'),
]
core.query().select(cols, group_by=rtab['is_old']).df()

2023-11-12 13:03:48,302 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-12 13:03:48,304 INFO sqlalchemy.engine.Engine SELECT count(*) AS count, min("Record".age) AS min, max("Record".age) AS max, sum("Record".age) / (count("Record".age) + 0.0) AS mean 
FROM "Record" GROUP BY "Record".is_old
2023-11-12 13:03:48,304 INFO sqlalchemy.engine.Engine [generated in 0.00179s] ()


,count,min,max,mean
0,5,25,47,36.8000000000
1,5,51,84,69.2000000000


In [42]:
# summarize age by decade
decade_expression = doctable.f.round(rtab['age'] / 10)
cols = [
    decade_expression.label('decade'),
    rtab['age'].count().label('count'),
    rtab['age'].min().label('min'),
    rtab['age'].max().label('max'),
    (rtab['age'].sum()/rtab['age'].count()).label('mean'),
]
core.query().select(cols, group_by=decade_expression).df()

2023-11-12 13:03:48,355 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-12 13:03:48,356 INFO sqlalchemy.engine.Engine SELECT round("Record".age / (? + 0.0)) AS decade, count("Record".age) AS count, min("Record".age) AS min, max("Record".age) AS max, sum("Record".age) / (count("Record".age) + 0.0) AS mean 
FROM "Record" GROUP BY round("Record".age / (? + 0.0))
2023-11-12 13:03:48,356 INFO sqlalchemy.engine.Engine [generated in 0.00148s] (10, 10)


,decade,count,min,max,mean
0,3.0,2,25,30,27.5000000000
1,4.0,2,40,42,41.0000000000
2,5.0,2,47,51,49.0000000000
3,6.0,1,58,58,58.0000000000
4,8.0,3,75,84,79.0000000000


#### Offset and Selecting Chunks

The `offset` parameter is used to pagify results into multiple queries - something that is particularly useful if the result set is too larget to fit into memory.

In [43]:
# get the three oldest individuals, offset by three
core.query().select(rtab.all_cols(), order_by=rtab['age'].desc(), limit=3, offset=3).df()

2023-11-12 13:03:48,405 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-12 13:03:48,406 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".age, "Record".is_old, "Record".name 
FROM "Record" ORDER BY "Record".age DESC
 LIMIT ? OFFSET ?
2023-11-12 13:03:48,406 INFO sqlalchemy.engine.Engine [generated in 0.00152s] (3, 3)


,id,age,is_old,name
0,10,58,1,user_9
1,5,51,1,user_4
2,9,47,0,user_8


In [44]:
for chunk in core.query().select_chunks(rtab.all_cols(), chunksize=3):
    print(chunk)

2023-11-12 13:03:48,454 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-12 13:03:48,454 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".age, "Record".is_old, "Record".name 
FROM "Record"
 LIMIT ? OFFSET ?
2023-11-12 13:03:48,455 INFO sqlalchemy.engine.Engine [generated in 0.00146s] (3, 0)
[(1, 84, 1, 'user_0'), (2, 75, 1, 'user_1'), (3, 42, 0, 'user_2')]
2023-11-12 13:03:48,456 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".age, "Record".is_old, "Record".name 
FROM "Record"
 LIMIT ? OFFSET ?
2023-11-12 13:03:48,456 INFO sqlalchemy.engine.Engine [cached since 0.00267s ago] (3, 3)
[(4, 25, 0, 'user_3'), (5, 51, 1, 'user_4'), (6, 40, 0, 'user_5')]
2023-11-12 13:03:48,457 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".age, "Record".is_old, "Record".name 
FROM "Record"
 LIMIT ? OFFSET ?
2023-11-12 13:03:48,458 INFO sqlalchemy.engine.Engine [cached since 0.003902s ago] (3, 6)
[(7, 78, 1, 'user_6'), (8, 30, 0, 'user_7'), (9, 47, 0, 'user_8')]
2023-11-